# Using TorchSharp to Generate Synthetic Data for a Regression Problem

This tutorial is based on a [PyTorch example](https://jamesmccaffrey.wordpress.com/2023/06/09/using-pytorch-to-generate-synthetic-data-for-a-regression-problem/) posted by James D. McCaffrey on his blog, ported to TorchSharp.

Note that we're taking some shortcuts in this example -- rather than writing the data set as a text file that can be loaded from any modeling framework, we're saving the data as serialized TorchSharp tensors. Is should be straight-forward to modify the tutorial to write the data sets as text, instead.

In [ ]:
#r "nuget: TorchSharp-cpu"

using TorchSharp;
using static TorchSharp.TensorExtensionMethods;

Neural networks can be used to generate data as well as train. The synthetic data can then be used to evaluate different models to see how well they can copy the behavior of the network used to produce the data.

First, we will create the model that will be used to generate the synthetic data. Later, we'll construct a second model that will be trained on the data the first model generates.

In [ ]:
class Net : torch.nn.Module<torch.Tensor,torch.Tensor>
{
    private torch.nn.Module<torch.Tensor,torch.Tensor> hid1;
    private torch.nn.Module<torch.Tensor,torch.Tensor> oupt;

    public Net(int n_in) : base(nameof(Net))
    {
        var h = torch.nn.Linear(n_in, 10);
        var o =  torch.nn.Linear(10,1);

        var lim = 0.80;
        torch.nn.init.uniform_(h.weight, -lim, lim);
        torch.nn.init.uniform_(h.bias, -lim, lim);
        torch.nn.init.uniform_(o.weight, -lim, lim);
        torch.nn.init.uniform_(o.bias, -lim, lim);

        hid1 = h;
        oupt = o;

        RegisterComponents();
    }
    public override torch.Tensor forward(torch.Tensor input)
    {
        using var _ = torch.NewDisposeScope();
        var z = torch.tanh(hid1.call(input));
        z = torch.sigmoid(oupt.call(z));
        return z.MoveToOuterDisposeScope();
    }
}

Now that we have our generative network, we can define the method to create the data set. If you compare this with the PyTorch code, you will notice that we're relying on TorchSharp to generate a whole batch of data at once, rather than looping. We're also using TorchSharp instead of Numpy for the noise-generation.

In [32]:
void CreateDataFile(Net net, int n_in, string fileName, int n_items)
{

    var x_lo = -1.0;
    var x_hi = 1.0;

    var X = (x_hi - x_lo) * torch.rand(new long[] {n_items, n_in}) + x_lo;

    torch.Tensor y;

    using (torch.no_grad()) {
        y = net.call(X);
    }

    // Add some noise in order to not make it too easy to train...
    y += torch.randn(y.shape) * 0.01;

    // Make sure that the output isn't negative.
    y = torch.where(y < 0.0, 0.01 * torch.randn(y.shape) + 0.01, y);

    // Save the data in two separate, binary files.
    X.save(fileName + ".x");
    y.save(fileName + ".y");
}

(torch.Tensor X, torch.Tensor y) LoadDataFile(string fileName)
{
    return (torch.Tensor.load(fileName + ".x"), torch.Tensor.load(fileName + ".y"));
}

In [ ]:
var net = new Net(6);

Create the data files.

In [ ]:
CreateDataFile(net, 6, "train.dat", 200);
CreateDataFile(net, 6, "test.dat", 40);

Load them again. This is just to demonstrate how to get the data from disk.

In [ ]:
var (X_train, y_train) = LoadDataFile("train.dat");
var (X_test, y_test) = LoadDataFile("test.dat");

Create another class, with slightly different logic, and train it on the generated data set.

In [ ]:
class Net2 : torch.nn.Module<torch.Tensor,torch.Tensor>
{
    private torch.nn.Module<torch.Tensor,torch.Tensor> hid1;
    private torch.nn.Module<torch.Tensor,torch.Tensor> oupt;

    public Net2(int n_in) : base(nameof(Net2))
    {
        hid1 = torch.nn.Linear(n_in, 5);
        oupt =  torch.nn.Linear(5,1);

        RegisterComponents();
    }
    public override torch.Tensor forward(torch.Tensor input)
    {
        using var _ = torch.NewDisposeScope();
        var z = torch.nn.functional.relu(hid1.call(input));
        z = torch.sigmoid(oupt.call(z));
        return z.MoveToOuterDisposeScope();
    }
}

We create an instance of the second network, choose a loss to use, and then we're ready to train it.

In [ ]:
var model = new Net2(6);

var loss = torch.nn.MSELoss();

A standard training loop. It ends with evaluating the trained model on the training set.

In [ ]:
var learning_rate = 0.01f;

Console.WriteLine(" initial loss = " + loss.forward(model.forward(X_train), y_train).item<float>().ToString());

var optimizer = torch.optim.SGD(model.parameters(), learning_rate);

for (int i = 0; i < 10000; i++) {
    // Compute the loss
    using var output = loss.forward(model.forward(X_train), y_train);

    // Clear the gradients before doing the back-propagation
    model.zero_grad();

    // Do back-progatation, which computes all the gradients.
    output.backward();

    optimizer.step();
}

Console.WriteLine(" final loss   = " + loss.forward(model.forward(X_train), y_train).item<float>());

The thing we're really curious about is how the second model does on the test set, which it didn't see during training. If the loss is significantly greater than the one from the training set, we need to train more, i.e. start another epoch. If the test set loss doesn't get closer to the training set loss with more epochs, we may need more data.

In [ ]:
loss.forward(model.forward(X_test), y_test).item<float>()